<a href="https://colab.research.google.com/github/anwitarajendra/Cron-like-Job-Scheduler/blob/main/code_for_job_scheduler.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import time
from datetime import datetime

LOG_FILE = "/content/mini_cron.log"

def simulate_daemon():
    print("Simulated daemon started. Logging every 10 seconds...")
    for _ in range(6):
        now = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        with open(LOG_FILE, 'a') as f:
            f.write(f"[{now}] Daemon is alive!\n")
        time.sleep(10)

simulate_daemon()


Simulated daemon started. Logging every 10 seconds...


In [ ]:
!cat /content/mini_cron.log


[2025-08-06 11:08:19] Daemon is alive!
[2025-08-06 11:08:29] Daemon is alive!
[2025-08-06 11:08:39] Daemon is alive!
[2025-08-06 11:08:49] Daemon is alive!
[2025-08-06 11:08:59] Daemon is alive!
[2025-08-06 11:09:09] Daemon is alive!


In [ ]:

job_file_content = """
*/1 * * * * echo "Hi Anwita"
0 14 * * * python3 greet.py
"""


with open("/content/jobs.txt", "w") as f:
    f.write(job_file_content.strip())

print("Created jobs.txt file with 2 scheduled tasks.")


Created jobs.txt file with 2 scheduled tasks.


In [ ]:
def parse_cron_line(line):
    parts = line.strip().split()
    if len(parts) < 6:
        return None
    schedule = parts[:5]
    command = ' '.join(parts[5:])
    return {"schedule": schedule, "command": command}

def read_jobs_file(file_path):
    jobs = []
    with open(file_path, 'r') as f:
        for line in f:
            if line.strip() == "" or line.startswith("#"):
                continue
            job = parse_cron_line(line)
            if job:
                jobs.append(job)
    return jobs


jobs = read_jobs_file("/content/jobs.txt")

for i, job in enumerate(jobs):
    print(f"Job {i+1}")
    print(f"  Schedule: {job['schedule']}")
    print(f"  Command:  {job['command']}\n")


Job 1
  Schedule: ['*/1', '*', '*', '*', '*']
  Command:  echo "Hi Anwita"

Job 2
  Schedule: ['0', '14', '*', '*', '*']
  Command:  python3 greet.py



In [ ]:
from datetime import datetime

def match_field(value, field):
    """
    value: int (current time unit like minute, hour)
    field: str (from schedule, like '*', '*/2', '5', etc.)
    """
    if field == '*':
        return True
    elif field.startswith('*/'):
        try:
            interval = int(field[2:])
            return value % interval == 0
        except:
            return False
    else:
        try:
            return value == int(field)
        except:
            return False

def should_run_job(job, now):
    schedule = job["schedule"]
    time_parts = [
        now.minute,
        now.hour,
        now.day,
        now.month,
        now.weekday()
    ]

    for t_val, sched_val in zip(time_parts, schedule):
        if not match_field(t_val, sched_val):
            return False
    return True


now = datetime.now()
print(f"Current Time: {now.strftime('%Y-%m-%d %H:%M')}")


for job in jobs:
    if should_run_job(job, now):
        print(f"RUN: {job['command']}")
    else:
        print(f"SKIP: {job['command']}")


Current Time: 2025-08-06 11:19
RUN: echo "Hi Anwita"
SKIP: python3 greet.py


In [ ]:
import subprocess
import time
from datetime import datetime

LOG_FILE = "/content/job_output.log"

def run_job(job):
    command = job['command']
    now = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

    try:

        result = subprocess.run(
            command, shell=True,
            capture_output=True,
            text=True
        )


        log_entry = f"\n[⏰ {now}] Command: {command}\n"
        log_entry += f"[STDOUT]\n{result.stdout}"
        if result.stderr:
            log_entry += f"[STDERR]\n{result.stderr}"
        log_entry += "-" * 40 + "\n"

        with open(LOG_FILE, 'a') as f:
            f.write(log_entry)

        print(f"Ran: {command}")
    except Exception as e:
        print(f"Failed to run: {command}")
        print(str(e))


print("Cron loop started (5 checks, 1 per minute)...\n")
for _ in range(5):
    now = datetime.now()
    print(f"Checking at {now.strftime('%H:%M')}")

    for job in jobs:
        if should_run_job(job, now):
            run_job(job)
        else:
            print(f"Skipping: {job['command']}")

    time.sleep(60)


Cron loop started (5 checks, 1 per minute)...

Checking at 11:21
Ran: echo "Hi Anwita"
Skipping: python3 greet.py
Checking at 11:22
Ran: echo "Hi Anwita"
Skipping: python3 greet.py
Checking at 11:23
Ran: echo "Hi Anwita"
Skipping: python3 greet.py
Checking at 11:24
Ran: echo "Hi Anwita"
Skipping: python3 greet.py
Checking at 11:25
Ran: echo "Hi Anwita"
Skipping: python3 greet.py


In [ ]:
!cat /content/job_output.log



[⏰ 2025-08-06 11:21:05] Command: echo "Hi Anwita"
[STDOUT]
Hi Anwita
----------------------------------------

[⏰ 2025-08-06 11:22:05] Command: echo "Hi Anwita"
[STDOUT]
Hi Anwita
----------------------------------------

[⏰ 2025-08-06 11:23:05] Command: echo "Hi Anwita"
[STDOUT]
Hi Anwita
----------------------------------------

[⏰ 2025-08-06 11:24:05] Command: echo "Hi Anwita"
[STDOUT]
Hi Anwita
----------------------------------------

[⏰ 2025-08-06 11:25:05] Command: echo "Hi Anwita"
[STDOUT]
Hi Anwita
----------------------------------------


In [ ]:
import sqlite3

# Connect (creates file if not exists)
conn = sqlite3.connect("/content/cron_history.db")
cursor = conn.cursor()

# Create table for job execution history
cursor.execute("""
CREATE TABLE IF NOT EXISTS job_runs (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    timestamp TEXT,
    command TEXT,
    output TEXT,
    error TEXT
)
""")
conn.commit()
conn.close()

print("Database and table created.")


Database and table created.


In [ ]:
import sqlite3


conn = sqlite3.connect("scheduler.db")
cursor = conn.cursor()


cursor.execute('''
CREATE TABLE IF NOT EXISTS jobs (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    command TEXT NOT NULL,
    schedule_type TEXT CHECK(schedule_type IN ('interval', 'daily')) NOT NULL,
    interval_minutes INTEGER,
    daily_time TEXT,
    last_run TEXT
)
''')

conn.commit()
conn.close()

print("Scheduler DB and table created successfully.")


Scheduler DB and table created successfully.


In [ ]:
from datetime import datetime

conn = sqlite3.connect("scheduler.db")
cursor = conn.cursor()


cursor.execute('''
INSERT INTO jobs (command, schedule_type, interval_minutes, last_run)
VALUES (?, 'interval', ?, ?)
''', ("print('Hello Anwita (interval)')", 2, datetime.now().isoformat()))


cursor.execute('''
INSERT INTO jobs (command, schedule_type, daily_time, last_run)
VALUES (?, 'daily', ?, ?)
''', ("print('Hello Anwita (daily)')", "16:30", datetime.now().isoformat()))

conn.commit()
conn.close()

print("Sample jobs inserted.")


Sample jobs inserted.


In [ ]:
conn = sqlite3.connect("scheduler.db")
cursor = conn.cursor()

cursor.execute("SELECT * FROM jobs")
rows = cursor.fetchall()

for row in rows:
    print(row)

conn.close()


(1, "print('🔁 Hello Anwita (interval)')", 'interval', 2, None, '2025-08-06T11:40:44.880899')
(2, "print('🕑 Hello Anwita (daily)')", 'daily', None, '16:30', '2025-08-06T11:40:44.882122')
(3, "print('Hello Anwita (interval)')", 'interval', 2, None, '2025-08-06T11:40:54.711526')
(4, "print('Hello Anwita (daily)')", 'daily', None, '16:30', '2025-08-06T11:40:54.712095')


In [ ]:
!pip install schedule


In [ ]:
import sqlite3
import time
import schedule
from datetime import datetime, timedelta

def run_job(command, job_id):
    print(f"Running Job ID {job_id} at {datetime.now().strftime('%H:%M:%S')}")
    try:
        exec(command)
    except Exception as e:
        print(f"Error running job {job_id}: {e}")


    conn = sqlite3.connect("scheduler.db")
    cursor = conn.cursor()
    cursor.execute("UPDATE jobs SET last_run = ? WHERE id = ?", (datetime.now().isoformat(), job_id))
    conn.commit()
    conn.close()

def check_and_schedule_jobs():
    conn = sqlite3.connect("scheduler.db")
    cursor = conn.cursor()
    cursor.execute("SELECT id, command, schedule_type, interval_minutes, daily_time, last_run FROM jobs")
    jobs = cursor.fetchall()
    conn.close()

    for job in jobs:
        job_id, command, schedule_type, interval_minutes, daily_time, last_run = job
        now = datetime.now()

        if schedule_type == "interval":
            if last_run:
                last_run_dt = datetime.fromisoformat(last_run)
                if now - last_run_dt >= timedelta(minutes=interval_minutes):
                    run_job(command, job_id)
            else:
                run_job(command, job_id)

        elif schedule_type == "daily":
            run_time = datetime.strptime(daily_time, "%H:%M").time()
            if last_run:
                last_run_dt = datetime.fromisoformat(last_run)
                already_ran_today = last_run_dt.date() == now.date()
                if not already_ran_today and now.time() >= run_time:
                    run_job(command, job_id)
            else:
                if now.time() >= run_time:
                    run_job(command, job_id)


print("Cron-like Job Scheduler started. Press STOP on Colab to exit.")
while True:
    check_and_schedule_jobs()
    time.sleep(30)


Cron-like Job Scheduler started. Press STOP on Colab to exit.
Running Job ID 1 at 11:45:34
🔁 Hello Anwita (interval)
Running Job ID 3 at 11:45:34
Hello Anwita (interval)
Running Job ID 1 at 11:47:34
🔁 Hello Anwita (interval)
Running Job ID 3 at 11:47:34
Hello Anwita (interval)
Running Job ID 1 at 11:49:34
🔁 Hello Anwita (interval)
Running Job ID 3 at 11:49:34
Hello Anwita (interval)
Running Job ID 1 at 11:51:34
🔁 Hello Anwita (interval)
Running Job ID 3 at 11:51:34
Hello Anwita (interval)
Running Job ID 1 at 11:53:34
🔁 Hello Anwita (interval)
Running Job ID 3 at 11:53:34
Hello Anwita (interval)
Running Job ID 1 at 11:55:34
🔁 Hello Anwita (interval)
Running Job ID 3 at 11:55:34
Hello Anwita (interval)
Running Job ID 1 at 11:57:34
🔁 Hello Anwita (interval)
Running Job ID 3 at 11:57:34
Hello Anwita (interval)
Running Job ID 1 at 11:59:34
🔁 Hello Anwita (interval)
Running Job ID 3 at 11:59:34
Hello Anwita (interval)
Running Job ID 1 at 12:01:34
🔁 Hello Anwita (interval)
Running Job ID 3 at

KeyboardInterrupt: 